#Set UP environment


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
! unzip -q "/content/drive/Shareddrives/UwU w what's dis /B2.onlybee.yolov8.zip" -d ./SavedModel
! ls ./SavedModel

Mounted at /content/drive
data.yaml  README.dataset.txt  README.roboflow.txt  test  train  valid


In [ ]:
# dataset_path = "/content/SavedModel/original BeeDataset_20201121"
dataset_path = "/content/SavedModel/train"
model_export_path = "/content/drive/Shareddrives/UwU w what's dis "
# labels_file = #example: C:\Users\Chuon\OneDrive\Tài liệu\GitHub\aiot-beehive\test\data.json
# export_path =

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from matplotlib import pyplot as plt
import cv2 as cv

# Introduction

In this notebook I will be using YOLO V7 for object detection. This notebook will be different from most of my other notebooks in that I will be cloning a github repo and working with the kaggle filesystem to make this functional.

I am using YoloV7 from https://github.com/WongKinYiu/yolov7 There are actually multiple claims to YOLO Version 7 due to seven being a lucky number. I have chosen this one principally because I find it the most user friendly of the claimants.

The repo I have chosen is an implementation of https://arxiv.org/abs/2207.02696

This notebook is very similar to my previous notebook with YOLO version 5 at: https://www.kaggle.com/code/taranmarley/yolo-v5-object-detection/
I am not entirely trying to be original though I have made some iterative improvements to the notebook beyond using a new version of YOLO

# Create the File System

In [ ]:
%cd ../
!mkdir tmp
%cd tmp

/
mkdir: cannot create directory ‘tmp’: File exists
/tmp


# Download YoloV7

In [ ]:
# Download YOLOv7
!git clone https://github.com/WongKinYiu/yolov7.git # clone repo
%cd yolov7
# Install dependencies
%pip install -qr requirements.txt  # install dependencies

%cd ../
import torch
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'yolov7'...
remote: Enumerating objects: 1197, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 1197 (delta 2), reused 3 (delta 1), pack-reused 1191
Receiving objects: 100% (1197/1197), 74.24 MiB | 33.11 MiB/s, done.
Resolving deltas: 100% (518/518), done.
/tmp/yolov7
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.4 MB/s eta 0:00:00
/tmp
Setup complete. Using torch 2.1.0+cu118 (Tesla T4)


In [ ]:
%cd yolov7

/tmp/yolov7


# Train the YoloV7 Model

In [ ]:
!python train.py --img 416 --batch 16 --epochs 200 --data /content/SavedModel/data.yaml  --weights 'yolov7.pt'

2023-12-06 08:16:03.449505: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-06 08:16:03.449551: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-06 08:16:03.449593: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-06 08:16:03.457347: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-06 08:16:04.451621: W tensorflow/compiler/

# Look at Training Results

In [ ]:
torch.load("/tmp/yolov7/runs/train/exp/weights/best.pt")

{'epoch': -1,
 'best_fitness': array([    0.76706]),
 'training_results': None,
 'model': Model(
   (model): Sequential(
     (0): Conv(
       (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
       (act): SiLU()
     )
     (1): Conv(
       (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
       (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
       (act): SiLU()
     )
     (2): Conv(
       (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
       (act): SiLU()
     )
     (3): Conv(
       (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
       (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_ru

In [29]:
img = cv.imread("runs/train/exp/results.png")
plt.figure(figsize=(15, 15))
plt.imshow(img)

# Model Validation Output

Let's look at how the model performed on a validation batch

In [30]:
img = cv.imread("/tmp/yolov7/runs/train/exp/test_batch0_pred.jpg")
plt.figure(figsize=(15, 15))
plt.imshow(img)

# Test the model

This will test the model we have created against an image it has not seen previously.

In [ ]:
img.shape

(1280, 1280, 3)

In [ ]:
img = cv.imread("/content/SavedModel/valid/images/d280598c-f000000021_png.rf.018947d803151cb3ec164dadf26fa8c7.jpg")

In [ ]:
!python detect.py --img 480 --conf-thres 0.3 --iou-thres 0.3 --source /content/SavedModel/test/images/1cef8014-6h5kkk7VfvaV1u2j4KGZ830KTiKqEC6y_jpeg.rf.f9097f97ee4ee83152ecda4d96f7175f.jpg --weights /tmp/yolov7/runs/train/exp/weights/best.pt

Namespace(weights=['/tmp/yolov7/runs/train/exp/weights/best.pt'], source='/content/SavedModel/test/images/1cef8014-6h5kkk7VfvaV1u2j4KGZ830KTiKqEC6y_jpeg.rf.f9097f97ee4ee83152ecda4d96f7175f.jpg', img_size=480, conf_thres=0.3, iou_thres=0.3, device='', view_img=False, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
YOLOR 🚀 v0.1-128-ga207844 torch 2.1.0+cu118 CPU

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 306 layers, 36479926 parameters, 6194944 gradients, 103.2 GFLOPS
 Convert model to T

In [ ]:
import argparse
import time
from pathlib import Path

import cv2
import torch
import torch.backends.cudnn as cudnn
from numpy import random

from models.experimental import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import check_img_size, check_requirements, check_imshow, non_max_suppression, apply_classifier, \
    scale_coords, xyxy2xywh, strip_optimizer, set_logging, increment_path
from utils.plots import plot_one_box
from utils.torch_utils import select_device, load_classifier, time_synchronized, TracedModel



In [ ]:
!pwd

/tmp/yolov7


In [ ]:

# def detect(save_img=False):
save_img = True # save inference images

weights = ['/tmp/yolov7/runs/train/exp/weights/best.pt']
source = "/content/SavedModel/test/images/d1ea6407-f000000017_png.rf.f292506a34acc49cfda2ebe1b05e097d.jpg"
imgsz=480
conf_thres=0.3
iou_thres=0.3
device=''
view_img=False
save_txt=False
save_conf=False
nosave=False
classes=None
agnostic_nms=False
augment=True
update=False
project='runs/detect'
name='exp'
exist_ok=False
no_trace=False

org_img = cv.imread(source)
# Directories
save_dir = "/tmp/yolov7/runs/detect/exp" # increment run
os.makedirs(save_dir, exist_ok=True)  # make dir

# Initialize
# set_logging()
device = select_device("cuda")
half = device.type != 'cpu'  # half precision only supported on CUDA

# Load model
model = attempt_load(weights, map_location=device)  # load FP32 model
stride = int(model.stride.max())  # model stride
imgsz = check_img_size(imgsz, s=stride)  # check img_size

if True:
    model = TracedModel(model, device, imgsz)

if half:
    model.half()  # to FP16

# Second-stage classifier
classify = False
if classify:
    modelc = load_classifier(name='resnet101', n=2)  # initialize
    modelc.load_state_dict(torch.load('weights/resnet101.pt', map_location=device)['model']).to(device).eval()

# Set Dataloader
dataset = LoadImages(source, img_size=imgsz, stride=stride)

# Get names and colors
names = model.module.names if hasattr(model, 'module') else model.names
colors = [[random.randint(0, 255) for _ in range(3)] for _ in names]

# Run inference
if device.type != 'cpu':
    model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))  # run once
old_img_w = old_img_h = imgsz
old_img_b = 1

t0 = time.time()
for path, img, im0s, vid_cap in dataset:
    img = torch.from_numpy(img).to(device)
    img = img.half() if half else img.float()  # uint8 to fp16/32
    img /= 255.0  # 0 - 255 to 0.0 - 1.0
    if img.ndimension() == 3:
        img = img.unsqueeze(0)

    # Warmup
    if device.type != 'cpu' and (old_img_b != img.shape[0] or old_img_h != img.shape[2] or old_img_w != img.shape[3]):
        old_img_b = img.shape[0]
        old_img_h = img.shape[2]
        old_img_w = img.shape[3]
        for i in range(3):
            model(img, augment=True)[0]

    # Inference
    t1 = time_synchronized()
    with torch.no_grad():   # Calculating gradients would cause a GPU memory leak
        pred = model(img, augment=True)[0]
    t2 = time_synchronized()

    # Apply NMS
    pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms)
    t3 = time_synchronized()

    # Apply Classifier
    if classify:
        pred = apply_classifier(pred, modelc, img, im0s)

    # Process detections
    for i, det in enumerate(pred):  # detections per image
        p, s, im0, frame = path, '', im0s, getattr(dataset, 'frame', 0)


        save_path = os.path.join(save_dir, os.path.basename(path))  # img.jpg
        # txt_path = str(save_dir / 'labels' / p.stem) + ('' if dataset.mode == 'image' else f'_{frame}')  # img.txt
        gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
        if len(det):
            # Rescale boxes from img_size to im0 size
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

            boundingbox = det.int().detach().cpu().numpy()
            print(boundingbox[0][:4])
            _1st_bb = boundingbox[0][:4]
            org_img[_1st_bb[1]:_1st_bb[3], _1st_bb[0]:_1st_bb[2]]
            cv.imwrite("test.png",org_img[_1st_bb[1]:_1st_bb[3], _1st_bb[0]:_1st_bb[2]])
            # Print results
            for c in det[:, -1].unique():
                n = (det[:, -1] == c).sum()  # detections per class
                s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string

            # Write results
            for *xyxy, conf, cls in reversed(det):
                if save_txt:  # Write to file
                    xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                    line = (cls, *xywh, conf) if opt.save_conf else (cls, *xywh)  # label format
                    # with open(txt_path + '.txt', 'a') as f:
                    #     f.write(('%g ' * len(line)).rstrip() % line + '\n')

                if save_img or view_img:  # Add bbox to image
                    label = f'{names[int(cls)]} {conf:.2f}'
                    plot_one_box(xyxy, im0, label=label, color=colors[int(cls)], line_thickness=1)

        # Print time (inference + NMS)
        print(f'{s}Done. ({(1E3 * (t2 - t1)):.1f}ms) Inference, ({(1E3 * (t3 - t2)):.1f}ms) NMS')

        # Stream results
        # if view_img:
        #     cv2.imshow(str(p), im0)
        #     cv2.waitKey(1)  # 1 millisecond

        # Save results (image with detections)
        if save_img:
            if dataset.mode == 'image':
                cv2.imwrite(save_path, im0)
                print(f" The image with the result is saved in: {save_path}")

if save_txt or save_img:
    s = f"\n{len(list(save_dir.glob('labels/*.txt')))} labels saved to {save_dir / 'labels'}" if save_txt else ''
    #print(f"Results saved to {save_dir}{s}")

print(f'Done. ({time.time() - t0:.3f}s)')





Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block


/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

[601  68 701 210]
14 Crawling things, Done. (12.1ms) Inference, (1.4ms) NMS
 The image with the result is saved in: /tmp/yolov7/runs/detect/exp/d1ea6407-f000000017_png.rf.f292506a34acc49cfda2ebe1b05e097d.jpg
Done. (0.235s)


In [ ]:
img = cv.imread("runs/detect/exp/1cef8014-6h5kkk7VfvaV1u2j4KGZ830KTiKqEC6y_jpeg.rf.f9097f97ee4ee83152ecda4d96f7175f.jpg")
plt.imshow(img)

# Conclusion

The results from this weren't better than my previous attempts with YOLO V5. This however isn't a scientific comparison of the two and any important project would be wise to use a variety of available approaches until they find the best one for that specific use case.

A real comparison of the techniques would need to take into account differences in parameter size, speed, memory usage, accuracy and a variety of further factors over many runs to determine which is working better with the dataset.

This however is just an exploration of YOLO V7 and I commend that they made it user friendly by providing documentation, keeping it similar to V5 and Google notebooks of it being used.